**Exercise 2 - XGBoost and SHAP for drought impact analysis**

In this exercise we - again - use data of drought-related indicators on agricultural fields, namely
- SPEI : Standarized Precipitation-Evaporation Index
- SMI : Soil Moisture Index
- LST/NDVI : Land Surface Temperature / Normalized Difference Vegetation Index
- AZL : Ackerzahl (soil quality)
- TWI : Topographic Wetness Index
- NFK : plant available water (*nutzbare Feldkapazität*)

In addition to the Python modules that you already know from last session, we now also need the machine learning libraries *scikit-learn*, *xgboost*, and *shap*. For data manipulation we use *numpy* and *pandas* instead of geopandas. The dataset is identical just without the geometry (.csv instread of .gpkg)

*scikit-learn* is abbreviated *sklearn* and contains a wide range of algorithms as well as data preparation routines, scoring metrics, and even model inspection techniques. Only a few functions are needed here, so we import them explicitly. More information on the included methods can be found on the website: https://scikit-learn.org


In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colormaps
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import xgboost as xgb
import shap

pd.set_option('display.max_colwidth', 255) # let pandas print full output of tables
os.listdir("data")

### Data preparation

The next two code blocks load the data and print the columns to check what it contains. Then data is reduced to the colums of interest for this exercise by typing the column names in double square brackets [[]]. Samples with missing values in any of the retained columns are excluded by *.dropna()*

*Crop* is a categorical variable of crop types. It can be used as such, but many algorithms are not designed to handle categorical variables. A workaround is to converted them to "one-hot encoding", meaning that each category is transferred to a separate binary variable.

In [ ]:
data = pd.read_csv("data/subset_random.csv")
print(data.shape, data.columns)


In [ ]:
# select columns of interest
data = data[["crop", "lstndvi_anom", "azl", "twi", "nfk", "SPEI_magnitude", "SMI_magnitude"]]

# remove all rows that contain missing values
data = data.dropna()

# convert crop type to binary columns (one-hot encoding)
data = pd.get_dummies(data, columns=["crop"])

# store the names of all crop columns as an object to easily access them
crop_cols = [col for col in data.columns if col.startswith('crop_')]

# print the first rows of the modified data table
data.head()

### Feature selection

The target vector of a regression is called **y**

The matrix of predictive features is called **X**

In the following setup, the crop health indicator *lstndvi_anom* is assigned as target and all other columns as predictive features. An implementation detail of XGBoost is that it requires input data in *numpy* format (as opposed to *scikit-learn* internal algorithms that can handle *pandas* objects)

In [ ]:
y = data["lstndvi_anom"]
X = data.drop("lstndvi_anom", axis="columns") # everything except lstndvi_anom

# Here would be an option to select only a subset of features
#X = data.drop("lstndvi_anom", axis="columns").drop(crop_cols, axis="columns")
#X_featureset1 = data[["SPEI_magnitude"]]
#X_featureset2 = data[["SPEI_magnitude", "azl"]]
# ...

# convert to numpy for xgb - and revert this later to get feature names in SHAP plots
Xnp = X.to_numpy()
ynp = y.to_numpy()

print(X.shape, y.shape)

### The Algorithm

*XGBoost* can be used for regression or classification. Here the *XGBoostRegressor* is used. It comes with many hyperparameters (i.e. parameters of the algorithm, as opposed to parameters of the fitted model). For these hyperparameters there are default settings that can be looked up by printing the object.

In [ ]:
algorithm = xgb.XGBRegressor()
algorithm

**One very important thing that is often overlooked by ML-beginners and even intermediate level articles & tutorials, is the algorithm-internal objective function.** ML models are crated from algorithms by optimizing a function in which performance is measured by a defined metric. That means the structure of the resulting model is adjusted to perform as good as possible on this very metric. When evaluating different models, it might make sense to compare them by a differnt metric that is related to the specifics of the intended application case. However, keep in mind that you might then be looking at metrics on which the model has not been optimized!

In the case of *XGBoostRegressor*, the default metric is the squared error

In [ ]:
algorithm.objective

### How **NOT** to do it

In principle we could now simply fit the algorithm to the data to obtain a model.

In [ ]:
model = algorithm.fit(X,y)

# print the R² score (coefficient of determination)
r2_score(y, model.predict(X))

This is overly optimistic!

### Sampling & Parametrization

A key concept of machine learning is that model skill needs to be assessed on "unseen" data, i.e. data not used during training. One reason for this is that strong algorithms can easily learn to perfectly fit complex training data - however, they might overfit rather than generalize, and consequently perform poor in application. Hyperparameters of the algorithms can be tuned to adjust the resulting model complexity (bias-variance tradeoff). Choosing good hyperparameters semi-automatically can be done by ranking models based on their performance on independent test data.

A common way to implement this is via cross-validation: data is subdivided into n sets, of which n-1 are used for training and the remaining 1 for testing. In the case of a 5-fold cross-validation that would mean 80% of data is used for training and 20% for testing, repeated 5 times.

However, it is good practise to assess the skill of the final model on yet another independent holdout set, that has not been used in the model tuning at all. Such a setup is termed nested cross-validation.

Note that in reality there might be stronger violations of the independence assumption than those caused by imperfect computational sampling. For example data from a small region or short period of time might not capture all dynamics of interest, no matter how the collected data is subdivided. In the case of this exercise, the data is from Brandenburg and covers the last 10 years. It does not contain all information on drought impacts "in general".


![nested cross validataion](nested_cv.png)

Typically these subsets are called 'training' and 'test', but for the sake of clarity let's call them 'holdout' and 'parametrization' here, as they refer to the outer loop of the sketched workflow.

In [ ]:
# the function train_test_split returns 4 objects. These can be stored by assigning 4 variables
X_parametrization, X_holdout, y_parametrization, y_holdout = train_test_split(Xnp, ynp, test_size=0.3)

# change the "test_size" parameter and check the dimensionality of the parametrizations (rows, columns)
print(X_parametrization.shape, y_parametrization.shape, X_holdout.shape, y_holdout.shape)

The next code block implements a 5-fold cross-validation during which different values for 2 parameters are tested. The resulting R² skill score on the "test" part of the data is used to create a ranking of parameter combinations.

In [ ]:
# defining some potential values for two parameters of interest
xgb_parameters = {
    'n_estimators':[10,50,100],
    'max_depth':[3,5,7]
    }

# parametrization internally using another data split, controlled by the parameter cv
gs = GridSearchCV(algorithm, param_grid=xgb_parameters, cv=5, scoring="r2")
gs_results = gs.fit(X_parametrization, y_parametrization)

# print the results in human-readible table format
pd.DataFrame({'rank': gs_results.cv_results_['rank_test_score'],
              'mean': gs_results.cv_results_['mean_test_score'],
              'sd'  : gs_results.cv_results_['std_test_score'],
              'params': gs_results.cv_results_['params']}).sort_values("rank")

The best model from this grid search is now re-trained using the entire parametrization subset, and then used to predict the holdout set. Note how much lower the score is than for the naive model fit on the full data.

In [ ]:
best_model = gs_results.best_estimator_.fit(X_parametrization, y_parametrization)
r2_score(y_holdout, best_model.predict(X_holdout))

**Regularization by early-stopping**

*XGBoost* has another way of adjusting model complexity: the maximum number of decision trees (boosted rounds) is controlled by the *n_estimators* parameter, but this maximum number might not be required. If there is no improvement of skill in a specified number of learning steps, the training can be stopped early, i.e. with a lower number of trees. This usually results in a less complex model, that might generalize better to unseen data, as tested on the holdout set.

In [ ]:
best_model.set_params(early_stopping_rounds=2)
best_model.fit(X_parametrization, y_parametrization, eval_set=[(X_holdout, y_holdout)])

In [ ]:
r2_score(y_holdout, best_model.predict(X_holdout))

### Automatization: wrap everything in a function

In [ ]:
def train(algorithm, param_grid, X, y, holdout=0.3, repetitions=10, ncv=5):
    """
    wrapper for repeated model fitting with random train test split, including the 
    computation of performance metrics on holdout set
    """

    # initialize empty lists to store the results
    cvscores = []
    holdoutscores = []
    paramlist = []
    estimatorlist = []

    # convert to numpy for xgb - and revert this later to get feature names in SHAP plots
    Xnp = X.to_numpy()
    ynp = y.to_numpy()
    
    for i in range(repetitions):
        print(i)

        # outer loop for validation
        X_parametrization, X_holdout, y_parametrization, y_holdout = train_test_split(Xnp, ynp, test_size=holdout)
        
        # inner loop to optimize the hyperparameters
        gs = GridSearchCV(algorithm, param_grid=param_grid, cv=ncv, scoring="r2")
        gs_results = gs.fit(X_parametrization, y_parametrization)

        # store the results in separate list
        cvscores.append(gs_results.best_score_)
        paramlist.append(gs_results.best_params_)
        estimatorlist.append(gs_results.best_estimator_)

        # re-train best estimator on full training set and compute score on holdout set
        best_model_of_iteration = gs_results.best_estimator_.fit(X_parametrization, y_parametrization)
        holdoutscores.append(r2_score(y_holdout, best_model_of_iteration.predict(X_holdout)))
    
    # take the best model of all iterations by scored on holdout set, 
    # active early stopping and fit on entire data of the inner loop
    best_model = estimatorlist[np.argmax(holdoutscores)]
    best_model.set_params(early_stopping_rounds=2)
    best_model.fit(X_parametrization, y_parametrization, eval_set=[(X_holdout, y_holdout)])
    n_stop = best_model.get_booster().num_boosted_rounds()
    best_model.set_params(n_estimators = n_stop, early_stopping_rounds=None)
    
    return(best_model, cvscores, holdoutscores, paramlist)

In [ ]:
# Initialize the algorithms with some fixed settings that will not be changed during parametrization
algorithm = xgb.XGBRegressor(booster='gbtree', tree_method='hist')

# define a set of parameter values to check
xgb_parameters = {
    'n_estimators':[100],
    'max_depth':[4,6,8,10],
    'gamma':[0, 0.1, 1],
    'subsample':[0.8],
    'colsample_bytree':[0.8],
    'learning_rate':[0.1]
    }

best_model, cvscores, holdoutscores, paramlist = train(algorithm, xgb_parameters, X, y)

In [ ]:
best_model

### Diagnostic plots

In [ ]:
def plotScores(holdoutscores, cvscores, plottitle):
    sns.distplot(holdoutscores, label="holdout", color="tomato") 
    sns.distplot(cvscores, label="cv", color="slateblue")
    plt.legend()
    plt.title(plottitle)

plotScores(holdoutscores, cvscores, "Model Setup 1")

In [ ]:
# plot predictions vs observations as scatter plot

plt.scatter(y_holdout, best_model.predict(X_holdout))
plt.xlim(-0.5,1.5)
plt.ylim(-0.5,1.5)

### Model inspection by SHAP

To figure out what exactly this model is doing, performance evaluation is not sufficient. SHAP plots visualize in which direction the feature values effect the prediction

In [ ]:
shapvals = shap.TreeExplainer(best_model).shap_values(X)
shap.summary_plot(shapvals, X, X.columns, max_display=10, cmap="coolwarm")


In [ ]:
mycmap = colormaps['coolwarm']
shap.dependence_plot("azl", shapvals, X, X.columns, cmap=mycmap,
                     interaction_index="crop_winter_wheat")

In [ ]:
shap.dependence_plot("crop_winter_wheat", shapvals, X, X.columns,
                     cmap=mycmap, interaction_index="azl")

In [ ]:
shap.dependence_plot("SPEI_magnitude", shapvals, X, X.columns, cmap=mycmap)

In [ ]:
shap.dependence_plot("SMI_magnitude", shapvals, X, X.columns, cmap=mycmap,
                     interaction_index="azl")